<h1>Creation of phenopackets from tabular data (individuals in columns)</h1>
<p>We will process <a href="https://pubmed.ncbi.nlm.nih.gov/18688868/" target="__blank">Hildebrand, et al. (2008) Autoimmune Disease in a DFNA6/14/38 Family Carrying a Novel Missense Mutation in WFS1</a></p>
<p>pyphetools provides a convenient way of extracting HPO terms from typical tables presented in supplemental material. Typical tables can have the individuals in columns or rows. In this case, we extract data from TABLE.</p>
<p>This note shows how to work through the table and set up the pyphetools encoder. The table was not originally available in the table, but constructed using the data in the publication</p>
<p>Users can work on one column at a time and then generate a collection of <a href="https://pubmed.ncbi.nlm.nih.gov/35705716/" target="__blank">GA4GH phenopackets</a> to represent each patient included in the original supplemental material. These phenopackets can then be used for a variety of downstream applications.</p>

In [1]:
import phenopackets as php
from google.protobuf.json_format import MessageToDict, MessageToJson
from google.protobuf.json_format import Parse, ParseDict
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
import os
import sys
sys.path.insert(0, os.path.abspath('../../../pyphetools'))
sys.path.insert(0, os.path.abspath('../../pyphetools'))
from pyphetools.creation import *
from pyphetools.creation.simple_column_mapper import try_mapping_columns
import numpy as np

<h2>Importing HPO data</h2>
<p>pyphetools uses the Human Phenotype Ontology (HPO) to encode phenotypic features. The recommended way of doing this is to ingest the hp.json file using HpoParser, which in turn creates an HpoConceptRecognizer object. </p>
<p>The HpoParser can accept a hpo_json_file argument if you want to use a specific file. If the argument is not passed, it will download the latext hp.json file from the HPO GitHub site and store it in a new subdirectory called hpo_data. It will not download the file if the file is already downloaded.</p>

In [2]:
parser = HpoParser()
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
metadata = MetaData(created_by="ORCID:0000-0002-5648-2155")
metadata.default_versions_with_hpo(version=hpo_version)

<h2>Importing the supplemental table</h2>
<p>Here, we use the pandas library to import this file (note that the Python package called openpyxl must be installed to read Excel files with pandas, although the library does not need to be imported in this notebook). pyphetools expects a pandas DataFrame as input, and users can choose any input format available for pandas include CSV, TSV, and Excel, or can use any other method to transform their input data into a Pandas DataFrame before using pyphetools.</p>

In [3]:
df = pd.read_excel('../../data/WFS1/PMID_18688868.xlsx')

In [4]:
df

,patient_id,II:2,III:1,III:3,IV:2,IV:4,V:2
0,Sex,female,female,female,female,female,male
1,Age,97,55,69,38,43,17
2,Variant,c.2576G>A,c.2576G>A,c.2576G>A,c.2576G>A,c.2576G>A,c.2576G>A
3,Low-frequency sensorineural hearing impairment,+,+,+,+,+,+
4,Progressive sensorineural hearing impairment,+,+,+,+,+,+
5,Graves disease,-,+,-,-,-,-
6,Crohn's disease,-,-,-,+,-,-


<h1>Converting to row-based format</h1>
<p>To use pyphetools, we need to have the individuals represented as rows (one row per individual) and have the items of interest be encoded as column names. The required transformations for doing this may be different for different input data, but often we will want to transpose the table (using the pandas <tt>transpose</tt> function) and set the column names of the new table to the zero-th row. After this, we drop the zero-th row (otherwise, it will be interpreted as an individual by the pyphetools code).</p>
<p>After this step is completed, the remaining steps to create phenopackets are the same as in the 
    <a href="http://localhost:8888/notebooks/notebooks/Create%20phenopackets%20from%20tabular%20data%20with%20individuals%20in%20rows.ipynb" target="__blank">row-based notebook</a>.</p>
    
Furthermore, for this specific case, there is a Count features row that we want dropped, so we filter out any row that does not have Patient in the first column.

In [5]:
dft = df.transpose()
dft.columns = dft.iloc[0]
dft.drop(dft.index[0], inplace=True)
dft.head()

patient_id,Sex,Age,Variant,Low-frequency sensorineural hearing impairment,Progressive sensorineural hearing impairment,Graves disease,Crohn's disease
II:2,female,97,c.2576G>A,+,+,-,-
III:1,female,55,c.2576G>A,+,+,+,-
III:3,female,69,c.2576G>A,+,+,-,-
IV:2,female,38,c.2576G>A,+,+,-,+
IV:4,female,43,c.2576G>A,+,+,-,-


Some column names might include spaces in front or after, and a couple of columns are subheadings and only contain NaNs, so lets correct that:

In [6]:
dft.columns = dft.columns.str.strip()
dft = dft.dropna(axis=1, how='all')
dft['patient_id'] = dft.index

<h2>Column mappers</h2>
<p>Please see the notebook "Create phenopackets from tabular data with individuals in rows" for explanations. In the following cell we create a dictionary for the ColumnMappers. Note that the code is identical except that we use the df.loc function to get the corresponding row data</p>

In [7]:
hpo_cr = parser.get_hpo_concept_recognizer()
column_mapper_d, col_not_found = try_mapping_columns(df=dft,
                                                    observed='+',
                                                    excluded='-',
                                                    hpo_cr=hpo_cr,
                                                    preview=True)

                                                          term    status
0  Low-frequency sensorineural hearing impairment (HP:0008573)  observed
1  Low-frequency sensorineural hearing impairment (HP:0008573)  observed
2  Low-frequency sensorineural hearing impairment (HP:0008573)  observed
3  Low-frequency sensorineural hearing impairment (HP:0008573)  observed
4  Low-frequency sensorineural hearing impairment (HP:0008573)  observed
5  Low-frequency sensorineural hearing impairment (HP:0008573)  observed
                                                        term    status
0  Progressive sensorineural hearing impairment (HP:0000408)  observed
1  Progressive sensorineural hearing impairment (HP:0000408)  observed
2  Progressive sensorineural hearing impairment (HP:0000408)  observed
3  Progressive sensorineural hearing impairment (HP:0000408)  observed
4  Progressive sensorineural hearing impairment (HP:0000408)  observed
5  Progressive sensorineural hearing impairment (HP:0000408)  o

In [8]:
print(col_not_found)

['Sex', 'Age', 'Variant', 'patient_id']


<h2>Variant Data</h2>
<p>The variant data (HGVS< transcript) is listed in the Variant (hg19, NM_015133.4) column.</p>

In [9]:
genome = 'hg38'
default_genotype = 'heterozygous'
transcript='NM_006005.3'
varMapper = VariantColumnMapper(assembly=genome,column_name='Variant', 
                                transcript=transcript, genotype=default_genotype)

<h1>Demographic data</h1>
<p>pyphetools can be used to capture information about age, sex, and individual identifiers. This information is stored in a map of "IndividualMapper" objects. Special treatment may be required for the indifiers, which may be used as the column names or row index.</p>

In [10]:
ageMapper = AgeColumnMapper.by_year('Age')
ageMapper.preview_column(dft['Age'])

,original column contents,age
0,97,P97Y
1,55,P55Y
2,69,P69Y
3,38,P38Y
4,43,P43Y
5,17,P17Y


In [11]:
sexMapper = SexColumnMapper(male_symbol='male', female_symbol='female', column_name='Sex')
sexMapper.preview_column(dft['Sex'])

,original column contents,sex
0,female,FEMALE
1,female,FEMALE
2,female,FEMALE
3,female,FEMALE
4,female,FEMALE
5,male,MALE


In [12]:
pmid = "PMID: 18688868"
encoder = CohortEncoder(df=dft, hpo_cr=hpo_cr, column_mapper_d=column_mapper_d, 
                        individual_column_name="patient_id", agemapper=ageMapper, sexmapper=sexMapper,
                       variant_mapper=varMapper, metadata=metadata,
                       pmid=pmid)
encoder.set_disease(disease_id='600965', label='DEAFNESS, AUTOSOMAL DOMINANT 6')

In [13]:
individuals = encoder.get_individuals()

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_006005.3%3Ac.2576G>A/NM_006005.3
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_006005.3%3Ac.2576G>A/NM_006005.3
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_006005.3%3Ac.2576G>A/NM_006005.3
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_006005.3%3Ac.2576G>A/NM_006005.3
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_006005.3%3Ac.2576G>A/NM_006005.3
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_006005.3%3Ac.2576G>A/NM_006005.3


In [14]:
i1 = individuals[0]
phenopacket1 = i1.to_ga4gh_phenopacket(metadata=metadata.to_ga4gh())
json_string = MessageToJson(phenopacket1)
print(json_string)

{
  "id": "II:2",
  "subject": {
    "id": "II:2",
    "timeAtLastEncounter": {
      "age": {
        "iso8601duration": "P97Y"
      }
    },
    "sex": "FEMALE"
  },
  "phenotypicFeatures": [
    {
      "type": {
        "id": "HP:0008573",
        "label": "Low-frequency sensorineural hearing impairment"
      },
      "onset": {
        "age": {
          "iso8601duration": "P97Y"
        }
      }
    },
    {
      "type": {
        "id": "HP:0000408",
        "label": "Progressive sensorineural hearing impairment"
      },
      "onset": {
        "age": {
          "iso8601duration": "P97Y"
        }
      }
    },
    {
      "type": {
        "id": "HP:0100647",
        "label": "Graves disease"
      },
      "excluded": true,
      "onset": {
        "age": {
          "iso8601duration": "P97Y"
        }
      }
    },
    {
      "type": {
        "id": "HP:0100280",
        "label": "Crohn's disease"
      },
      "excluded": true,
      "onset": {
        "age": {
   

In [15]:
output_directory = "../../phenopackets/WFS1/"
encoder.output_phenopackets(outdir=output_directory)

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_006005.3%3Ac.2576G>A/NM_006005.3
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_006005.3%3Ac.2576G>A/NM_006005.3
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_006005.3%3Ac.2576G>A/NM_006005.3
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_006005.3%3Ac.2576G>A/NM_006005.3
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_006005.3%3Ac.2576G>A/NM_006005.3
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_006005.3%3Ac.2576G>A/NM_006005.3
Wrote 6 phenopackets to ../../phenopackets/WFS1/
